# SUM vizualisation

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import panel as pn
import numpy as np
import pyvista as pv
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf

pn.extension('vtk')
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'

In [ ]:
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
from torch_points3d.datasets.segmentation.sum import SUMDataset
from torch_points3d.datasets.segmentation import IGNORE_LABEL

## Load SUM dataset

In [ ]:
DIR = os.path.dirname(os.getcwd())
dataset_options = OmegaConf.create()
dataset_options.data = OmegaConf.load(os.path.join(DIR,'conf/data/segmentation/sum.yaml'))

In [ ]:
dataset_options.data.dataroot = os.path.join(DIR,"data")
dataset = SUMDataset(dataset_options.data)
print(dataset)

## Visualise the data

In [ ]:
OBJECT_COLOR = [
        [233, 229, 107],  #'ceiling' .-> .yellow
        [95, 156, 196],  #'floor' .-> . blue
        [179, 116, 81],  #'wall'  ->  brown
        [241, 149, 131],  #'beam'  ->  salmon
        [81, 163, 148],  #'column'  ->  bluegreen
        [77, 174, 84],  #'window'  ->  bright green
        [108, 135, 75],  #'door'   ->  dark green
        [41, 49, 101],  #'chair'  ->  darkblue
        [79, 79, 76],  #'table'  ->  dark grey
        [223, 52, 52],  #'bookcase'  ->  red
        [89, 47, 95],  #'sofa'  ->  purple
        [81, 109, 114],  #'board'   ->  grey
        [233, 233, 229],  #'clutter'  ->  light grey
        [0, 0, 0],  # unlabelled .->. black
    ]
def buil_cmap():
    mapping = np.linspace(0, len(OBJECT_COLOR), 256)
    newcolors = np.zeros((256, 3))
    for i, color in enumerate(OBJECT_COLOR):
        newcolors[mapping >= i-0.5] = np.asarray(color) / 255.
    return ListedColormap(newcolors)
cmap = buil_cmap()

In [ ]:
def load_random_data(event):
    i = np.random.randint(0, len(dataset.train_dataset))
    sample = dataset.train_dataset[i]
    xyz = sample.pos.numpy()
    pl = pv.Plotter(notebook=True)
    point_cloud = pv.PolyData(xyz)
    point_cloud['label'] = sample.y.numpy()
    pl.add_points(point_cloud,cmap=cmap, clim=[0, len(OBJECT_COLOR)]) 
    
    pan.object = pl.ren_win
    return point_cloud

In [ ]:
pl = pv.Plotter(notebook=True)
pan = pn.panel(pl.ren_win, sizing_mode='scale_both', aspect_ratio=1,orientation_widget=True,)

In [ ]:
button = pn.widgets.Button(name='Load new model', button_type='primary')
button.on_click(load_random_data)

In [ ]:
dashboard = pn.Row(
    pn.Column('## S3DIS visualiser',button),
    pan
)

In [ ]:
dashboard

## Validation dataset

In [ ]:
i = 3
area = dataset.val_dataset[i]
xyz = area.pos.numpy()
pl = pv.Plotter(notebook=True)
point_cloud = pv.PolyData(xyz)
point_cloud['label'] = area.y.numpy()
pl.add_points(point_cloud,cmap=cmap, clim=[0, len(OBJECT_COLOR)]) 
pl.show()

## Train dataset

In [ ]:
i = 3
area = dataset.train_dataset._datas[i]
xyz = area.pos.numpy()
pl = pv.Plotter(notebook=True)
point_cloud = pv.PolyData(xyz)
point_cloud['label'] = area.y.numpy()
pl.add_points(point_cloud,cmap=cmap, clim=[0, len(OBJECT_COLOR)]) 
pl.show()

In [ ]:
i = 3
area = dataset.trainval_dataset._datas[i]
xyz = area.pos.numpy()
pl = pv.Plotter(notebook=True)
point_cloud = pv.PolyData(xyz)
point_cloud['label'] = area.y.numpy()
pl.add_points(point_cloud,cmap=cmap, clim=[0, len(OBJECT_COLOR)]) 
pl.show()

# Dataset verification

In [ ]:
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
from torch_points3d.datasets.segmentation.sum import SUMPointCloudDataset
from torch_geometric.data import DataLoader

In [ ]:
dataset = SUMPointCloudDataset(
    "../data/sum",
    length = 100,
    cloud_size = 4092,
    process_workers=42
)

In [ ]:
loader = DataLoader(dataset, batch_size=100, shuffle=False)

In [ ]:
from torch import unique

while True:
    batch = next(iter(loader))
    value, count = unique(batch.y, sorted=True, return_counts=True)
    print(value)
    print(count)

In [5]:
from plyfile import PlyData, PlyElement
import numpy as np

dataset = SUMPointCloudDataset(
    "../data/sum",
    length = 1,
    cloud_size = -1,
    process_workers=42
)

data = dataset[0]

vertex = np.empty(data.num_nodes,dtype=[('x', '>f8'), ('y', '>f8'), ('z', '>f8'), ('nx', '>f8'), ('ny', '>f8'), ('nz', '>f8'), ('red', 'uint8'), ('green', 'uint8'), ('blue', 'uint8'), ('l', 'uint8')])
vertex["x"] = data.pos[:,0].numpy()
vertex["y"] = data.pos[:,1].numpy()
vertex["z"] = data.pos[:,2].numpy()
vertex["nx"] = data.normal[:,0].numpy()
vertex["ny"] = data.normal[:,1].numpy()
vertex["nz"] = data.normal[:,2].numpy()
vertex["red"] = (data.rgb[:,0].numpy()*255).astype(np.uint8)
vertex["green"] = (data.rgb[:,1].numpy()*255).astype(np.uint8)
vertex["blue"] = (data.rgb[:,2].numpy()*255).astype(np.uint8)
vertex["l"] = data.y.numpy().astype(np.uint8)
el = PlyElement.describe(vertex, 'vertex')
PlyData([el]).write('test.ply')

/home/ggrzeczkowicz/.local/anaconda3/envs/pyTorch-Points3D/lib/python3.8/site-packages/torch_geometric/data/dataset.py:145: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, make sure to sure to delete '../data/sum/processed' first
  warnings.warn(
